In [1]:
import keras
from keras.models import Model
from keras.layers import Input, Dense, Embedding, LSTM

In [2]:
#recieving a sequence of 100 int, between 1 - 10000.
main_input = Input(shape=(100,), dtype='int32', name='main_input')

In [3]:
#the embedding layer will encode the input sequence
x = Embedding(output_dim = 512, input_dim = 10000, input_length = 100)(main_input)

In [4]:
#lstm will transform the vector sequence into a single vector contains info about the entire sequence
lstm_out = LSTM(32)(x)

In [5]:
# this allows Lstm and Embedding to be trained smoothly even though the main loss will be much higher in the model
auxiliary_input = Input(shape=(5,), name = 'aux_input')

In [6]:
auxiliary_output = Dense(1, activation = 'sigmoid', name = 'aux_output')(lstm_out)

In [7]:
x = keras.layers.concatenate([lstm_out, auxiliary_input])

In [8]:
# stacking denses connected on top
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)
x = Dense(64, activation = 'relu')(x)

In [9]:
main_output = Dense(1, activation = 'sigmoid', name = 'main_output')(x)

In [15]:
model = Model(inputs = [main_input, auxiliary_input], outputs = [main_output, auxiliary_output])

In [16]:
model.compile(optimizer='rmsprop',
             loss={'main_output': 'binary_crossentropy', 'aux_output': 'binary_crossentropy'},
             loss_weights={'main_output': 1., 'aux_output': 0.2})

In [17]:
model.summary()

Model: "model_1"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
main_input (InputLayer)         [(None, 100)]        0                                            
__________________________________________________________________________________________________
embedding (Embedding)           (None, 100, 512)     5120000     main_input[0][0]                 
__________________________________________________________________________________________________
lstm (LSTM)                     (None, 32)           69760       embedding[0][0]                  
__________________________________________________________________________________________________
aux_input (InputLayer)          [(None, 5)]          0                                            
____________________________________________________________________________________________

In [18]:
import numpy as np

In [19]:
np.random.seed(0)

In [20]:
main_data = np.round(np.abs(np.random.rand(12,100) * 100))
aux_data = np.random.randn(12,5)

In [21]:
main_out = np.random.randn(12,1)
aux_out = np.random.randn(12,1)

In [22]:
model.fit({'main_input': main_data, 'aux_input': aux_data},
         {'main_output': main_out, 'aux_output': aux_out},
         epochs= 10,
         batch_size = 32)

Epoch 1/10
1/1 [==============================] - 3s 3s/step - loss: 0.7866 - main_output_loss: 0.6474 - aux_output_loss: 0.6961
Epoch 2/10
1/1 [==============================] - 0s 67ms/step - loss: 0.6734 - main_output_loss: 0.5436 - aux_output_loss: 0.6491
Epoch 3/10
1/1 [==============================] - 0s 60ms/step - loss: 0.5850 - main_output_loss: 0.4624 - aux_output_loss: 0.6132
Epoch 4/10
1/1 [==============================] - 0s 64ms/step - loss: 0.4972 - main_output_loss: 0.3835 - aux_output_loss: 0.5689
Epoch 5/10
1/1 [==============================] - 0s 72ms/step - loss: 0.4009 - main_output_loss: 0.2954 - aux_output_loss: 0.5275
Epoch 6/10
1/1 [==============================] - 0s 59ms/step - loss: 0.2857 - main_output_loss: 0.1918 - aux_output_loss: 0.4694
Epoch 7/10
1/1 [==============================] - 0s 66ms/step - loss: 0.1393 - main_output_loss: 0.0579 - aux_output_loss: 0.4068
Epoch 8/10
1/1 [==============================] - 0s 59ms/step - loss: -0.0569 - main

In [23]:
model.predict({'main_input': main_data, 'aux_input': aux_data})

[array([[0.00317547],
        [0.04662156],
        [0.31343287],
        [0.00446612],
        [0.00417751],
        [0.5495355 ],
        [0.0015097 ],
        [0.32988083],
        [0.00316578],
        [0.49968585],
        [0.46098498],
        [0.00966024]], dtype=float32),
 array([[0.30854642],
        [0.2322318 ],
        [0.1884751 ],
        [0.2551781 ],
        [0.23924723],
        [0.4189574 ],
        [0.25544542],
        [0.28533685],
        [0.14196303],
        [0.28825143],
        [0.4701362 ],
        [0.3393785 ]], dtype=float32)]